In [ ]:
!pip install openai -q
!pip install yFinance -q
import openai
openai.api_key = "" #replace with api key

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.7 MB/s eta 0:00:00


Just run this single code block to start querying

In [ ]:
import datetime
from datetime import date
import json
import logging
import operator
import sys

import openai
import yfinance as yf

logging.basicConfig(level=logging.WARNING, format="%(asctime)s %(message)s")
date_today = date.today()

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)


def get_price(symbol: str, date: str) -> float:
    logger.info(f"Calling get_price with {symbol=} and {date=}")

    history = yf.download(
        symbol, start=date, period="1d", interval="1d", progress=False
    )

    return history["Close"].iloc[0].item()

def calculate(a: float, b: float, op: str) -> float:
    logger.info(f"Calling calculate with {a=}, {b=} and {op=}")

    return getattr(operator, op)(a, b)


get_price_metadata = {
    "name": "get_price",
    "description": "Get closing price of a financial instrument on a given date",
    "parameters": {
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Ticker symbol of a financial instrument",
            },
            "date": {
                "type": "string",
                "description": "Date in the format YYYY-MM-DD",
            },

        },
        "required": ["symbol", "date"],
    },
}

calculate_metadata = {
    "name": "calculate",
    "description": "General purpose calculator",
    "parameters": {
        "type": "object",
        "properties": {
            "a": {
                "type": "number",
                "description": "First entry",
            },
            "b": {
                "type": "number",
                "description": "Second entry",
            },
            "op": {
                "type": "string",
                "enum": ["mul", "add", "truediv", "sub"],
                "description": "Binary operation",
            },

        },
        "required": ["a", "b", "op"],
    },
}

messages = [{"role": "user", "content": input()},

            {"role": "system", "content": f"You are an helpfful financial investor who overlooks the performace of stocks. Today is {date_today} .For Indian stocks add the suffix .NS to them. /nExample: Zomato's ticker will be ZOMATO.NS and for all the other companies it will just be their ticker symbol. /nExample: Apple's ticker symbol will still be APPL, there is no need to .NS as suffix for such companies that are based outside of India"}]

while True:
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        temperature=0,
        messages=messages,
        functions=[get_price_metadata, calculate_metadata],
    )
    message = response["choices"][0]["message"]
    messages.append(message)

    if "function_call" not in message:
        break

    # call custom functions
    function_name = message["function_call"]["name"]
    kwargs = json.loads(message["function_call"]["arguments"])

    if function_name == "get_price":
        output = str(get_price(**kwargs))
    elif function_name == "calculate":
        output = str(calculate(**kwargs))
    else:
        raise ValueError

    messages.append(
        {"role": "function", "name": function_name, "content": output}
    )


print(messages[-1]["content"])


Zomato


INFO:__main__:Calling get_price with symbol='ZOMATO.NS' and date='2023-07-07'


The closing price of Zomato (ZOMATO.NS) on July 7, 2023, was ₹73.25.
